# 새 섹션

# **밑에 칸 만들어 놨어요!!**

# 참고사항
# FinanceDataReader 을 통해 해외주식을 읽어올 경우 'Change' 칼럼이 없습니다.
# 시계열 데이터 결측값 처리 df이름.interpolate(inplace=True)

In [1]:
!pip install Konlpy
!pip install Wordcloud
!pip install finance-datareader
!pip install yfinance
!pip install googletrans==3.1.0a0
!pip install TextBlob
!pip install Afinn
import FinanceDataReader as fdr
import pandas as pd
import yfinance as yf
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from bs4 import BeautifulSoup
from lxml import etree
from urllib import parse
import requests
import urllib.request
import urllib.parse
import numpy as np
import time
from konlpy.tag import Kkma
from tqdm import tqdm
from wordcloud import WordCloud
from afinn import Afinn



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 KB 31.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 KB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 KB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 KB 7.0

# 금준호

In [8]:
import FinanceDataReader as fdr
import pandas as pd
import datetime
import requests
from bs4 import BeautifulSoup
import time
from tqdm import tqdm
from googletrans import Translator
from afinn import Afinn
import numpy as np
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import re
from nltk.tokenize import word_tokenize


samsung = fdr.DataReader('005930', start = '2021-01-01', end = '2021-12-31')
samsung.isnull().value_counts()
samsung = samsung.drop(['Open','High','Low'], axis = 1)
print(samsung)
type(samsung)



date_samsung = samsung.index
close_samsung = samsung.Close
change_samsung = samsung.Change

samsung_df = pd.DataFrame({'date':date_samsung, 'close' : close_samsung, 
                         'change' : change_samsung*100},
             columns=['date', 'close', 'change',])

samsung_df
len(samsung_df)

samsung_df = samsung_df.reset_index(drop=True)
samsung_df.to_csv('samsung_df', index=False)

print(samsung_df)



############# ##########################크롤링 #######################

stock_code = "005930"
start_date = "20210101"
end_date = "20211231"
titles = []
dates = []
previews = []

current_date = datetime.datetime.strptime(start_date, "%Y%m%d")
end_date = datetime.datetime.strptime(end_date, "%Y%m%d")

while current_date <= end_date:
    if current_date.weekday() >= 5 or current_date.strftime("%Y-%m-%d") in holidays:
        current_date += datetime.timedelta(days=1)
        continue

    for page in tqdm(range(1, 4)):
        url = f"https://finance.naver.com/news/news_search.nhn?rcdate=&q={stock_code}&sm=all.basic&pd=4&stDate={current_date.strftime('%Y%m%d')}&enDate={current_date.strftime('%Y%m%d')}&page={page}"
        response = requests.get(url)
        soup = BeautifulSoup(response.text, "html.parser")
        articles = soup.find_all("dd", class_="articleSubject")
        
        if len(articles) == 0:
            break

        for article in articles:
            title = article.a.text.strip()
            preview_tag = article.find_next_sibling("dt")
            preview = preview_tag.text.strip() if preview_tag else ""
            titles.append(title)
            dates.append(current_date.strftime("%Y-%m-%d"))
            previews.append(preview)

        time.sleep(0.45)

    current_date += datetime.timedelta(days=1)





print(news.isnull().values.any()) #null값이 있는지 체크
news_data = news.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(news_data.isnull().values.any()) # Null 값이 존재하는지 확인

news_data
#############################################################################



# 저장, 불러오기 
news_data.to_csv('news_csv_2021', index=False)
news_df = pd.read_csv('news_csv_2021', encoding = 'utf-8')
news_df



# 분석하기
news_df
samsung_df

len(samsung_df)
news_df["date"].nunique()

samsung_df["date"].nunique()


news_df['date'] = pd.to_datetime(news_df['date'])



# 겹치는 요일만 합치기 
merged_df = pd.merge(samsung_df, news_df, on='date')
merged_df



merged_df = merged_df.groupby(['date', 'change'])['Title'].apply(' '.join).reset_index()
merged_df



# KOR -> ENG 번역, 감성점수 구하기 
def preprocess_text(text):
    text = re.sub(r'[^a-zA-Z0-9\s]', ' ', text)
    words = word_tokenize(text)
    stop_words = set(['i', 'the', 'a', 'is', 'that'])
    words = [w for w in words if (w not in stop_words) and (len(w) > 2)]
    text = ' '.join(words)
    return text



def translate_and_sentiment(df): # 번역하고 감성점수 구하기 
    translator = Translator()
    afinn = Afinn(language='en')

    eng_news = []
    sentiments = []
    for title in tqdm(df['Title']):
        try:
            eng_title = translator.translate(title, dest='en').text
            eng_title = preprocess_text(eng_title) # 전처리 함수 적용 
            eng_news.append(eng_title)
            sentiment = afinn.score(eng_title)
            sentiments.append(sentiment)
        except Exception as e:
            print(e)
            eng_news.append('')
            sentiments.append(0)

    df['Translated Title'] = eng_news
    df['Sentiment'] = sentiments

    return df

merged_df = translate_and_sentiment(merged_df) # 함수적용 


'''
AFINN은  -5에서 +5 사이의 숫자로 점수가 매겨진 단어 목록  0 ~ 100사이의 결과가 나옴 
한계점 : 긍정, 부정 기준을 잡기가 애매함
아직 활성화된 한국어 감성사전이 없어서 번역해서 감성분석을 해야하는게 부정확함
같은 단어가 긍정일수도 부정일수도 있음 
주식 시장의 단어사전이 아닌 일반 단어사전을 사용해서 정확도가 낮음 
'''

mean_sentiment = merged_df['Sentiment'].mean()
median_sentiment = merged_df['Sentiment'].median()
print(mean_sentiment) # 43.891129032258064
print(median_sentiment) # 45


print(merged_df[merged_df['Sentiment'] >= 40])
print(merged_df[merged_df['Sentiment'] < 40])


merged_df['score'] = np.where((merged_df['Sentiment'] >= 40) & (merged_df['change'] >= 0), 'TRUE', 'FALSE')
merged_df

merged_df.info()
merged_df
merged_df = merged_df.drop(['Title','Translated Title'], axis = 1)


merged_df.to_csv("merged_df", index = False)
merged_df = pd.read_csv('merged_df', encoding = 'utf-8')




len(merged_df[merged_df['score'] == True]) # True 개수
'''
   실제로 변동율이 올랐을때 감성점수가 긍정이거나
   변동율이 떨어졌을때 감성점수가 부정인 행 -> 95개 
'''


len(merged_df[merged_df['score'] == False]) # False 개수  -> 153개

# 정확도 구하기 
acc = len(merged_df[merged_df['score'] == True])/(len(merged_df[merged_df['score'] == True])+len(merged_df[merged_df['score'] == False]))
print(round(acc, 2))#  약 38%


corr = pearsonr(merged_df['change'],merged_df['Sentiment'])
corr
# PearsonRResult(statistic=-0.05333039172616527, pvalue=0.40304231375608907)

sns.regplot(x='change', y='Sentiment', data=merged_df)
plt.title(f'Pearson Correlation: {corr:.2f}')
plt.show()

            Close    Volume    Change
Date                                 
2021-01-04  83000  38655276  0.024691
2021-01-05  83900  35335669  0.010843
2021-01-06  82200  42089013 -0.020262
2021-01-07  82900  32644642  0.008516
2021-01-08  88800  59013307  0.071170
...           ...       ...       ...
2021-12-24  80500  12086380  0.007509
2021-12-27  80200  10783368 -0.003727
2021-12-28  80300  18226325  0.001247
2021-12-29  78800  19794795 -0.018680
2021-12-30  78300  14236700 -0.006345

[248 rows x 3 columns]
          date  close    change
0   2021-01-04  83000  2.469136
1   2021-01-05  83900  1.084337
2   2021-01-06  82200 -2.026222
3   2021-01-07  82900  0.851582
4   2021-01-08  88800  7.117008
..         ...    ...       ...
243 2021-12-24  80500  0.750939
244 2021-12-27  80200 -0.372671
245 2021-12-28  80300  0.124688
246 2021-12-29  78800 -1.867995
247 2021-12-30  78300 -0.634518

[248 rows x 3 columns]


 33%|███▎      | 1/3 [00:02<00:05,  2.51s/it]


KeyboardInterrupt: ignored

# 지윤님


In [ ]:

high_close_ss = ssdf.loc[ssdf['close'].idxmax()]
# date      2021-01-11 00:00:00
# close                   91000
# volume               90306177
# change               2.477477
# Name: 2021-01-11 00:00:00, dtype: object

low_close_ss = ssdf.loc[ssdf['close'].idxmin()]
# date       2022-09-29 00:00:00
# close                    52600
# volume                13882080
# change               -0.567108
# weekday                      3
# Name: 2022-09-29 00:00:00, dtype: object 

highest30 = ssdf.nlargest(n=30,columns='close',keep='all')
highest30.sort_values(by=['date'])

#                  date  close    volume    change  weekday
# Date                                                     
# 2021-01-05 2021-01-05  83900  35335669  1.084337        1 
# 2021-01-08 2021-01-08  88800  59013307  7.117008        4 (==> 2021-01-01 - 2021-01-08)
# 2021-01-11 2021-01-11  91000  90306177  2.477477        0
# 2021-01-12 2021-01-12  90600  48682416 -0.439560        1
# 2021-01-13 2021-01-13  89700  36068848 -0.993377        2
# 2021-01-14 2021-01-14  89700  26393970  0.000000        3
# 2021-01-15 2021-01-15  88000  33431809 -1.895206        4
# 2021-01-18 2021-01-18  85000  43227951 -3.409091        0 (==> 2021-01-04 - 2021-01-18)
# 2021-01-19 2021-01-19  87000  39895044  2.352941        1
# 2021-01-20 2021-01-20  87200  25211127  0.229885        2
# 2021-01-21 2021-01-21  88100  25318011  1.032110        3
# 2021-01-22 2021-01-22  86800  30861661 -1.475596        4
# 2021-01-25 2021-01-25  89400  27258534  2.995392        0
# 2021-01-26 2021-01-26  86700  33178936 -3.020134        1
# 2021-01-27 2021-01-27  85600  26423070 -1.268743        2 (==> 2021-01-01 - 2021-01-27)
# 2021-02-02 2021-02-02  84400  26302077  1.686747        1
# 2021-02-03 2021-02-03  84600  22112205  0.236967        2 (==> 2021-01-28 - 2021-02-03) ==> 2021-01-01 - 2021-02-03★
# 2021-02-15 2021-02-15  84200  23529706  3.186275        0
# 2021-02-16 2021-02-16  84900  20483100  0.831354        1 ==> 2021-02-10 - 2021-02-16★
# 2021-02-25 2021-02-25  85300  34155986  4.024390        3 (==> 2021-02-19 - 2021-02-25)
# 2021-03-03 2021-03-03  84000  19882132  0.478469        2 (==> 2021-02-25 - 2021-03-03) ==> 2021-02-19 - 2021-03-03★
# 2021-04-02 2021-04-02  84800  22997538  2.291918        4 
# 2021-04-05 2021-04-05  85400  16255990  0.707547        0
# 2021-04-06 2021-04-06  86000  19042023  0.702576        1
# 2021-04-07 2021-04-07  85600  17885538 -0.465116        2
# 2021-04-08 2021-04-08  84700  19244259 -1.051402        3 
# 2021-04-13 2021-04-13  84000  15238206  0.961538        1
# 2021-04-14 2021-04-14  84000  13794652  0.000000        2
# 2021-04-15 2021-04-15  84100  16377412  0.119048        3
# 2021-04-16 2021-04-16  83900  15429522 -0.237812        4
# 2021-04-20 2021-04-20  83900  15521965  0.720288        1 ==> 2021-03-27 - 2021-04-20★

highest30_dates = [["2021-01-01", "2021-02-03"], ["2021-02-10", "2021-02-16"], ["2021-02-19", "2021-03-03"], ["2021-03-27", "2021-04-20"]]

lowest30 = ssdf.nsmallest(n=30,columns='close',keep='all')
lowest30.sort_values(by=['date'])
#                  date  close    volume    change  weekday
# Date                                                     
# 2022-07-01 2022-07-01  56200  24982097 -1.403509        4
# 2022-07-06 2022-07-06  56400  16820461 -1.398601        2 ==> 2022-06-25 - 2022-07-06★
# 2022-09-07 2022-09-07  56000  17384854 -1.926445        2
# 2022-09-08 2022-09-08  55600  18000016 -0.714286        3 (==> 2022-09-01 - 2022-09-08)
# 2022-09-15 2022-09-15  56000  11664819 -1.408451        3 
# 2022-09-16 2022-09-16  56200  13456503  0.357143        4
# 2022-09-19 2022-09-19  56400  12278653  0.355872        0
# 2022-09-20 2022-09-20  55800  14041465 -1.063830        1
# 2022-09-21 2022-09-21  55300  11863700 -0.896057        2
# 2022-09-22 2022-09-22  54400  12786510 -1.627486        3
# 2022-09-23 2022-09-23  54500  10555964  0.183824        4
# 2022-09-26 2022-09-26  53900  15008449 -1.100917        0
# 2022-09-27 2022-09-27  54200  16631289  0.556586        1
# 2022-09-28 2022-09-28  52900  19991129 -2.398524        2
# 2022-09-29 2022-09-29  52600  13882080 -0.567108        3
# 2022-09-30 2022-09-30  53100  22503249  0.950570        4
# 2022-10-04 2022-10-04  55200  19148435  3.954802        1
# 2022-10-05 2022-10-05  56000  22458970  1.449275        2
# 2022-10-06 2022-10-06  56300  14944446  0.535714        3
# 2022-10-07 2022-10-07  56200  16886813 -0.177620        4
# 2022-10-11 2022-10-11  55400  21437877 -1.423488        1
# 2022-10-12 2022-10-12  55800  18408910  0.722022        2
# 2022-10-13 2022-10-13  55200  13784602 -1.075269        3
# 2022-10-14 2022-10-14  56300  12924326  1.992754        4
# 2022-10-17 2022-10-17  56600  13641878  0.532860        0
# 2022-10-18 2022-10-18  56500  14820702 -0.176678        1
# 2022-10-19 2022-10-19  55800  12660652 -1.238938        2
# 2022-10-20 2022-10-20  55500  17107653 -0.537634        3
# 2022-10-21 2022-10-21  55900  11456060  0.720721        4 ==> 2022-09-01 - 2022-10-21★
# 2022-12-28 2022-12-28  56600  14665410 -2.581756        2
# 2022-12-29 2022-12-29  55300  11295935 -2.296820        3 ==> 2022-12-22 - 2022-12-29★

lowest30_dates = [["2022-06-25", "2022-07-06"], ["2022-09-01", "2022-10-21"], ["2022-12-22", "2022-12-29"]]


highest30_dates = [["2021-01-01", "2021-02-03"], ["2021-02-10", "2021-02-16"], ["2021-02-19", "2021-03-03"], ["2021-03-27", "2021-04-20"]]
lowest30_dates = [["2022-06-25", "2022-07-06"], ["2022-09-01", "2022-10-21"], ["2022-12-22", "2022-12-29"]]

highest30_pages = getLastPage(highest30_dates) #[485, 43, 108, 259]
lowest30_pages = getLastPage(lowest30_dates) #[119, 532, 80]


HighestSentiment = sentimentAnalysis(engHighestnews)      #len(positive[1]) 1725   len(negative[1]) 593  sum(positive[1]), sum(negative[1]) 360.99340414347535 -76.37707624716543 ==> 1.2683510001400577
LowestSentiment = sentimentAnalysis(engLowestnews)        #len(positive[1]) 897    len(negative[1]) 615  sum(positive[1]), sum(negative[1]) 172.7414368171508 -85.3758926397677   ==> 1.9772261300911067

Highest30Sentiment= sentimentAnalysis(engHighest30news)   #len(positive[1]) 3358   len(negative[1]) 1432 sum(positive[1]), sum(negative[1]) 734.1611423206502 -187.01524124651795 ==> 1.341801411432267
Lowest30Sentiment = sentimentAnalysis(engLowest30news)    #len(positive[1]) 1099   len(negative[1]) 692  sum(positive[1]), sum(negative[1]) 245.64260060640422 -106.08840184583909==> 1.7601949836554633





# 란님


In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Wed Mar 22 16:54:53 2023

@author: ITWILL
"""

import FinanceDataReader as fdr # 주식
import pandas as pd
import matplotlib.pyplot as plt # 그래프 그리기
import urllib.request as req  # url 가져오기 
from bs4 import BeautifulSoup
import re
import datetime # 전후 n일 뽑기위해
from konlpy.tag import Okt # class - 형태소 
from wordcloud import WordCloud # class - 단어 구름 시각화 
from collections import Counter # 단어 카운트 


# 하이브
hybe = fdr.DataReader('352820', start='2022-02-01', end='2023-03-22')
len(hybe) # 281
C_hybe = hybe.Close
high_hybe = hybe[hybe['Close']==max(hybe.Close)]
low_hybe = hybe[hybe['Close']==min(hybe.Close)]
high_hybe
low_hybe

# 에스엠
sm = fdr.DataReader('041510', start='2022-02-01', end='2023-03-22')
len(sm)  # 281
C_sm = sm.Close
high_sm = sm[sm['Close']==max(sm.Close)]
low_sm = sm[sm['Close']==min(sm.Close)]
high_sm
low_sm


# jyp
jyp = fdr.DataReader('035900', start='2022-02-01', end='2023-03-22')
len(jyp) # 281
C_jyp = jyp.Close


# yg
yg = fdr.DataReader('122870', start='2022-02-01', end='2023-03-22')
len(yg) # 281
C_yg = yg.Close

# 큐브
cube = fdr.DataReader('182360', start='2022-02-01', end='2023-03-22')
len(cube) # 281
C_cube = cube.Close


# 그래프
plt.plot(C_hybe,'red')
plt.plot(C_sm,'blue')
plt.plot(C_jyp,'green')
plt.plot(C_yg,'black')
plt.plot(C_cube,'pink')
plt.legend(['hybe','sm','jyp','yg','cube'])

plt.show()


# 상한가,하한가,변화량(가장큰,가장작은) 날짜 찾는 함수
def stock_func(stock_num):
    
    stock = fdr.DataReader(stock_num, start='2022-02-01', end='2023-03-22')
    len(stock) # 281
    high = stock[stock['Close']==max(stock.Close)]
    low = stock[stock['Close']==min(stock.Close)]
    big_change = stock[stock['Change']==max(stock.Change)]
    small_change = stock[stock['Change']==min(stock.Change)]
    
    # 상한가 date
    Hdate_index = high.index # date 뽑기
    Hdate_time = str(Hdate_index[0]) # 날짜만 뽑기 전 str형 변환
    Hdate = Hdate_time.split()[0] # 시간 제외, 날짜만 뽑기
    
    # 하한가 date
    Ldate_index = low.index
    Ldate_time = str(Ldate_index[0])
    Ldate = Ldate_time.split()[0]
    
    # 변화량최대 date
    Bdate_index = big_change.index
    Bdate_time = str(Bdate_index[0])
    Bdate = Bdate_time.split()[0]
    
    # 변화량최소 date
    Sdate_index = small_change.index
    Sdate_time = str(Sdate_index[0])
    Sdate = Sdate_time.split()[0]

    
    return Hdate,Ldate,Bdate,Sdate

hybe = stock_func('352820') # %C7%CF%C0%CC%BA%EA
hybe # ('2022-04-01', '2022-10-13', '2022-11-11', '2022-11-11')

sm = stock_func('041510') # %BF%A1%BD%BA%BF%A5
sm # ('2023-03-08', '2022-05-12')
jyp =stock_func('035900') # jyp
jyp # ('2023-03-08', '2022-02-15')
yg = stock_func('122870')
yg # ('2022-04-01', '2022-11-21')
cube = stock_func('182360')
cube # ('2022-03-28', '2022-06-23')


# 웹크롤링 함수
def crawler_func(date,stock_name,pages=3):  
    
    title =[]
    subnews = []
    
    for page in range(1, pages+1) : 
        
        # 1) url 구성 
        url = f"https://finance.naver.com/news/news_search.naver?rcdate=&q={stock_name}&x=0&y=0&sm=all.basic&pd=4&stDateStart={date}&stDateEnd={date}&page={page}" 
        # 2) url 요청          
        res = req.urlopen(url)
        data = res.read()
        
        # 3) html 파싱
        src = data.decode('euc-kr') 
        html = BeautifulSoup(src, 'html.parser')
        
        # 4) a 태그 수집                  
        step1 = html.select('#contentarea_left > div > dl > dd > a') 
                         # //*[@id="contentarea_left"]/div[2]/dl/dd[1]/a
        
        
        title_list = []
        
        for t in step1 :
            title_step2 = str(t.text).strip()
            title_list.append(title_step2)
        title.extend(title_list)
        
        
        # 5) 보조기사 수집
        news_step1 = html.select('#contentarea_left > div > dl > dd')
        print(news_step1)
        
        subnews_list =[]
        
        for n in news_step1 :
            sub = str(n.text)
            sub = re.sub('[\t\n]','',sub)
            subnews_list.append(sub)
        subnews.extend(subnews_list)
  
    return title,subnews


Hhybe_title = crawler_func(hybe[0],'%C7%CF%C0%CC%BA%EA')
Hhybe_title[0]
Lhybe_title = crawler_func(hybe[1],'%C7%CF%C0%CC%BA%EA')
Lhybe_title



# 명사 저장
okt = Okt()
nouns_word = [] # 명사 저장 

for sent in Hhybe_title[0] : 
    for noun in okt.nouns(sent) : # 문장(1개) -> 명사, 서수 제외(okt)
            nouns_word.append(noun)

nouns_word


# 변화량 플마 7일 찾는 함수
def stock7_func(stock_num):
    
    stock = fdr.DataReader(stock_num, start='2022-02-01', end='2023-03-22')
    len(stock) # 281
    big_change = stock[stock['Change']==max(stock.Change)]
    small_change = stock[stock['Change']==min(stock.Change)]
    
    # 변화량 max date
    big_change_index = big_change.index # date 뽑기
    Bbefore = big_change_index - datetime.timedelta(days=7)
    Bafter = big_change_index + datetime.timedelta(days=7)
    Bbefore = str(Bbefore[0]) # 날짜만 뽑기 전 str형 변환    
    Bbefore = Bbefore.split()[0] # 시간 제외, 날짜만 뽑기
    Bafter = str(Bafter[0]) # 날짜만 뽑기 전 str형 변환
    Bafter = Bafter.split()[0] # 시간 제외, 날짜만 뽑기
    
    # 변화량 min date
    small_change_index = small_change.index # date 뽑기
    Sbefore = small_change_index - datetime.timedelta(days=7)
    Safter = small_change_index + datetime.timedelta(days=7)
    Sbefore = str(Sbefore[0]) # 날짜만 뽑기 전 str형 변환
    Sbefore = Sbefore.split()[0] # 시간 제외, 날짜만 뽑기
    Safter = str(Safter[0]) # 날짜만 뽑기 전 str형 변환
    Safter = Safter.split()[0] # 시간 제외, 날짜만 뽑기
    
    
    
    return Bbefore,Bafter,Sbefore,Safter 

hybe = stock7_func('352820') # %C7%CF%C0%CC%BA%EA
hybe


def crawler7_func(Bdate,Adate,stock_name,pages=200):  
    
    title =[]
    subnews = []
    
    for page in range(1, pages+1) : 
        
        # 1) url 구성 
        url = f"https://finance.naver.com/news/news_search.naver?rcdate=&q={stock_name}&x=0&y=0&sm=all.basic&pd=4&stDateStart={Bdate}&stDateEnd={Adate}&page={page}" 
        # 2) url 요청          
        res = req.urlopen(url)
        data = res.read()
        
        # 3) html 파싱
        src = data.decode('euc-kr') 
        html = BeautifulSoup(src, 'html.parser')
        
        # 4) a 태그 수집                  
        step1 = html.select('#contentarea_left > div > dl > dd > a') 
                         # //*[@id="contentarea_left"]/div[2]/dl/dd[1]/a
        
        
        title_list = []
        
        for t in step1 :
            title_step2 = str(t.text).strip()
            title_list.append(title_step2)
        title.extend(title_list)
        
        
        # 5) 보조기사 수집
        news_step1 = html.select('#contentarea_left > div > dl > dd')
        # print(news_step1)
        
        subnews_list =[]
        
        for n in news_step1 :
            sub = str(n.text)
            sub = re.sub('[\t\n]','',sub)
            #sub = sub.split('...') # 보조기사에서 언론사,기사날짜 제거 : ...으로 구별 불가능
            #sub = sub[0] 
            #print(sub)
            subnews_list.append(sub)
        subnews.extend(subnews_list)
        #print(news_list)
    #news = pd.DataFrame({"Title": title, "SubNews": subnews}) : title,subnews 수 다름
        #print(url)
        
    
    return title,subnews

Bigchange_hybe = crawler7_func(hybe[0],hybe[1],'%C7%CF%C0%CC%BA%EA')
len(Bigchange_hybe[0]) # 80


# title 단어

okt = Okt()
nouns_word = [] # 명사 저장 

for sent in Bigchange_hybe[0] : 
    for noun in okt.nouns(sent) : # 문장(1개) -> 명사, 서수 제외(okt)
            nouns_word.append(noun)

nouns_word
len(nouns_word) # 592

wc = {}

for word in nouns_word :
     wc[word] = wc.get(word, 0) + 1 # 단어를 wc의 키값으로 만들어줌
    
print(wc)

# subnews 단어

nouns_word2 = [] # 명사 저장 

for sent in Bigchange_hybe[1] : 
    for noun in okt.nouns(sent) : # 문장(1개) -> 명사, 서수 제외(okt)
            nouns_word2.append(noun)

nouns_word2
len(nouns_word2) # 2143

wc2 = {}

for word in nouns_word2 :
     wc2[word] = wc.get(word, 0) + 1
    
print(wc2)

###################################################### title과 subnews 합쳐야하나?

nouns_word_final = nouns_word + nouns_word2
print(nouns_word_final)
len (nouns_word_final) # 2735
wcf = {}

for word in nouns_word_final :
     wcf[word] = wcf.get(word, 0) + 1
    
print(wcf)

count = Counter(wcf) # Counter object 생성
del count['등'] # 불용어 처리 : 의미 없는 명사 제거

# top10 단어 선정
top10_word = count.most_common(10)  
print(top10_word) # [('단어',빈도수)]

'''
[('하이브', 186), ('주가', 40), ('콘텐츠', 36), ('목표', 34), ('방탄소년단', 28), ('경제', 28), 
('실적', 27), ('하향', 27), ('증권', 20), ('등', 18)]
'등' 제외하고 다시

-> [('하이브', 186), ('주가', 40), ('콘텐츠', 36), ('목표', 34), ('방탄소년단', 28), ('경제', 28), 
    ('실적', 27), ('하향', 27), ('증권', 20), ('투자', 17)]

'''

# 단어와 출현빈도수 추출 
words = []
cnt = []

for (w,c) in top10_word :
    words.append(w)
    cnt.append(c)

print(words)
print(cnt)


plt.rcParams['font.family'] = 'Malgun Gothic' # 한글은 맑은고딕체

# 막대그래프
plt.bar(words, cnt) # x축, y축
plt.title('news word count') # 차트 제목 
plt.show()  # 차트 보이기 


# word cloud
wc = WordCloud(font_path='C:/Windows/Fonts/malgun.ttf',
          width=500, height=400,
          max_words=100,max_font_size=150,
          background_color='white')

wc_result = wc.generate_from_frequencies(dict(wcf))

plt.imshow(wc_result)
plt.axis('off') # 축 눈금 감추기 
plt.show()

'''
불용어때문에 함수XXXXXXXXX

def wordcloud_func(stock_title):  # word cloud 함수

    okt = Okt()
    
    # title 단어
    nouns_word = [] # 명사 저장 
    for word in stock_title[0] : 
        for noun in okt.nouns(word) : # 문장(1개) -> 명사, 서수 제외(okt)
                nouns_word.append(word)

    # subnews 단어
    nouns_word2 = [] # 명사 저장 

    for sent in stock_title[1] : 
        for noun in okt.nouns(sent) : # 문장(1개) -> 명사, 서수 제외(okt)
                nouns_word2.append(noun)

    nouns_word_final = nouns_word + nouns_word2
    wcf = {}

    for word in nouns_word_final :
         wcf[word] = wcf.get(word, 0) + 1
        
    print(wcf)

    count = Counter(wcf) # Counter object 생성

    # top10 단어 선정
    top10_word = count.most_common(10)  
    print(top10_word) # [('단어',빈도수)]

    # 단어와 출현빈도수 추출 
    words = []
    cnt = []

    for (w,c) in top10_word :
        words.append(w)
        cnt.append(c)

    print(words)
    print(cnt)


    # word cloud

    wc = WordCloud(font_path='C:/Windows/Fonts/malgun.ttf',
              width=500, height=400,
              max_words=100,max_font_size=150,
              background_color='white')

    wc_result = wc.generate_from_frequencies(dict(wcf))

    plt.imshow(wc_result)
    plt.axis('off') # 축 눈금 감추기 
    plt.show()
    
    return top10_word
'''


'''
# 두호님 변화량 N개 뽑기
def top_N_changes(finance_data, N):
    finance_data['Abs_Change'] = finance_data['Change'].abs()
    top_N = finance_data.sort_values(by='Abs_Change', ascending=False).head(N)
    # Drop the 'Abs_Change' column
    top_N = top_N.drop('Abs_Change', axis=1)
    return top_N

print(top_N_changes(samsung, 10))

'''
# 미국주식
# WarnerMusciGruop
wmg = fdr.DataReader('WMG', start = '2020-11', end = '2023')
wmg

wmg.isnull() # 결측치 처리 
wmg.isnull().value_counts()

wmg = wmg.dropna()   # 결측치 있는 행 삭제
wmg = wmg.dropna(axis = 1) # 결측치 있는 열 삭제 

wmg.dropna(inplace = True)

df_wmg = pd.DataFrame(wmg)
print(df_wmg)
type(df_wmg)
# df_wmg['Close'].plot()




# Nasdaq
nasdaq = fdr.DataReader('IXIC', start = '2020-11', end = '2023')
nasdaq

nasdaq.isnull() # 결측치 처리 
nasdaq.isnull().value_counts()

nasdaq = nasdaq.dropna()   # 결측치 있는 행 삭제
nasdaq = nasdaq.dropna(axis = 1) # 결측치 있는 열 삭제 

nasdaq.dropna(inplace = True)

df_nasdaq = pd.DataFrame(nasdaq)
print(df_nasdaq)
type(df_nasdaq)
# df_nasdaq['Adj Close'].plot()

close_wmg = df_wmg.Close
close_nasdaq=df_nasdaq['Adj Close']
fig, bx1 = plt.subplots()
bx1.plot(close_wmg, color='b')
bx2 = bx1.twinx()
bx2.plot(close_nasdaq, color='r')
plt.show()

######## 삼성 변화률 top30 

dates = ['2022-03-07','2022-04-13','2022-04-29','2022-06-13','2022-07-07',
         '2022-07-18','2022-09-13','2022-10-04','2022-10-24','2022-10-28',
         '2022-10-31','2022-11-08','2022-11-11','2022-11-30','2022-12-02']
B_dates = ['2022-03-05','2022-04-11','2022-04-27','2022-06-11','2022-07-05',
         '2022-07-16','2022-09-11','2022-10-02','2022-10-22','2022-10-26',
         '2022-10-29','2022-11-06','2022-11-09','2022-11-28','2022-11-30']

samsung_title = []
samsung_sub = []

for i in range(0,29) :
    samsung = crawler7_func(B_dates[i],dates[i],'%BB%EF%BC%BA')
    samsung_title.append(samsung[0])
    samsung_sub.append(samsung[1])

samsung_news = samsung_title + samsung_sub

def clean_text(texts) :
    from re import sub # 함수 임포트

    # 단계1 : 소문자 변경
    # texts_re = [t.lower() for t in texts]

    #단계2 : 숫자 제거 : 변수 = [실행문(method/func) for 변수 in 열거형객체]
    texts_re = [sub('[0-9]', '', t) for t in texts]

    # 단계3 : 문장부호 제거
    punc_str = '[\',.?!:;\"…]'
    texts_re2 = [sub(punc_str, '', t) for t in texts_re]

    # 단계4 : 특수문자 제거
    spec_str = '[@#$%^&◇△|*()-_\[\]]'
    texts_re3 = [sub(spec_str, '', t) for t in texts_re2]

    # 단계5 : 공백(white space) 제거
    texts_re4 = [' '.join(t.split()) for t in texts_re3]

    return texts_re4

#################################### 기사번역 -> 단어추출

clean_news_title = clean_text(samsung_news) 
print(clean_news_title)


import googletrans # 기사번역을 위해
translator = googletrans.Translator() # 객체생성

news_title = []
for title in clean_news_title:
    try : 
        result =  translator.translate(title, dest='en')
    #print(result.text)
        news_title.append(result.text)
    except : 
        pass
news_title


clean_eng_news = clean_text(news_title) 


from konlpy.tag import Okt # class - 형태소 
okt = Okt()

import nltk # Natural Laguage Toolkit
nltk.download('punkt') # nltk data download
from nltk.tokenize import word_tokenize # 문장 -> 단어 token
nltk.download("stopwords") # 라이브러리 다운
from nltk.corpus import stopwords #  불용어제거

news_word = []
for title in clean_eng_news : 
    result = word_tokenize(title)
    news_word.extend(result)
print(news_word)
len(news_word) # 1324

stopwords.words('english') # 영어 불용어 모아둔 것
 
news_stopwords = []   
for w in news_word :
    if w not in stopwords.words('english') :
            news_stopwords.append(w)
    else :
        pass
print(news_stopwords)




# 두호님


In [ ]:
!pip install Konlpy
!pip install Wordcloud
!pip install finance-datareader
!pip install yfinance
##################### 네이버 증권뉴스 크롤링 #########################
import urllib.request as req  # url 가져오기 
from bs4 import BeautifulSoup # html parsing 도구
#import pandas as pd # 날짜 생성 
from re import findall, match, sub, search  # 정규표현식 

# 0. 검색하고자하는 종목 검색 : 삼성전자, 네이버
tickers = ['%BB%EF%BC%BA%C0%FC%C0%DA','%B3%D7%C0%CC%B9%F6']

# 1. 수집 기간 날짜 설정 
dates = [["2023-03-21","2023-03-22"],["2023-03-22","2023-03-23"],[""]]

# 각 페이지당 url마지막 페이지 번호 추출 함수
def getLastPage(ticker, date) :
    last_page_list = []
    for date in dates :  
        try:
           print('검색중 : ', ticker)
           # 1) url 구성 
           url = f"https://finance.naver.com/news/news_search.naver?rcdate=&q={ticker}&x=0&y=0&sm=all.basic&pd=4&stDateStart={date[0]}&stDateEnd={date[1]}"
           
           # 2) url 요청          
           res = req.urlopen(url)
           data = res.read()
           
           # 3) html 파싱
           #src = data.decode('utf-8') 
           html = BeautifulSoup(data, 'html.parser')
           select_page = html.select('td[class="pgRR"] > a')
           
           page_a= select_page[0].get('href')
           
           # = 기준 split 해서 0,1 로 찾아도 된다
           search_page = search("page=.*", page_a)
           page_num = search(r'\d+', search_page.group())
           page_num = int(page_num.group()) 
           
           last_page_list.append(page_num)
         
        except Exception as e:
            print('에러 :',e)
    return last_page_list

page_list = []
for ticker in tickers : 
    page_list.append(getLastPage(ticker, dates))
    
#print(page_list)


# 2. Crawler 함수(페이지, 검색날짜, 페이지 리스트) 
def crawler_func(ticker, date, page_list) :  
    print('수집중 날짜:', dates)
    print(date[0],date[1])
    day_news = []
    for page_num in page_list :    
        for curr_page in range(1, page_num + 1) : 
            print(curr_page)
            # 1) url 구성 
            url = f"https://finance.naver.com/news/news_search.naver?rcdate=&q={ticker}&x=0&y=0&sm=all.basic&pd=4&stDateStart={date[0]}&stDateEnd={date[1]}&page={curr_page}"
            
            # 2) url 요청          
            res = req.urlopen(url)
            data = res.read()
            
            # 3) html 파싱
            html = BeautifulSoup(data, 'html.parser')
            
            # 4) a 태그 수집    
            links = html.select('dl[class="newsList"] > dd') # 1) dd tag 수집
            #print(links)                
            page_news = [] # 1 페이지 news 저장
            for i,link in enumerate(links) :
                print(i,link)
                #print(link)
                news = str(link.text).strip() # 문장 끝 불용어 제거 
                page_news.append(news)  
                #print(page_news)         
            day_news.extend(page_news) # {day}의 모든 페이지 루프 돌고 day_news에 저장   
            #print(page_news)       
    return day_news
        
            
# 3. 클로러 함수 호출 
zipped = list(zip(tickers, dates, page_list))
#print(zipped)
'''
for x,y,z in zipped:
    print(x,y,z)
    type(x)
    type(y)
'''
articles_list = {}
for ticker, date, page in zipped : 
    articles_list[ticker] = crawling_news = crawler_func(ticker, date, page)
    
# 크롤링한 데이터 전처리
def clean_text(texts) :        
    punc_str = '[,.?!:;]'
    texts_re = [sub(punc_str, '', st) for st in texts]
    spec_str = '[@#$%^&*()]'
    texts_re2 = [sub(spec_str, '', st) for st in texts_re]
    texts_re3 = [sub('[a-z]', '', st) for st in texts_re2]
    texts_re4 = [sub('\s+', ' ', st) for st in texts_re3] #형식 1 
    return texts_re4

clean_articles ={}
for article in articles_list.items():
    #print(article[0])
    clean_articles[article[0]] = clean_text(article[1])

print(clean_articles)




##############################################################################
# 삼성 vs 애플 & 나스닥 지수

import FinanceDataReader as fdr
import pandas as pd
import yfinance as yf
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# 날짜 설정
start_date = '2022-01-01'
end_date = '2022-12-31'

# 주식데이터 가져오기
apple = fdr.DataReader('AAPL', start_date, end_date)
samsung = fdr.DataReader('005930', start_date, end_date)
nasdaq = fdr.DataReader('IXIC', start_date, end_date)

# 종가 가져오기. 결측값들은 Interpolation 을 이용하여 채우기.
apple_close = apple['Close']
#apple_close.interpolate(inplace=True) # Linear interpolation
samsung_close = samsung['Close']
samsung_close.interpolate(inplace=True)

# 환율 교정(애플미국달러 * 한국달려환율)
usd_krw_exchange_rate = fdr.DataReader('USD/KRW', start_date, end_date)['Close']
#print(usd_krw_exchange_rate)
apple_close = apple_close * usd_krw_exchange_rate
apple_close.interpolate(inplace=True)
#print(apple_close)

# Combine the data into a single DataFrame
data = pd.concat([apple_close, samsung_close], axis=1)
data.columns = ['Apple', 'Samsung']

# Create the subplots
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(15, 10))

# Plot the data for Apple and Samsung on the left subplot
ax1.plot(data.index, data['Apple'], label='Apple', color = 'r')
ax1.plot(data.index, data['Samsung'], label='Samsung', color = 'b')
ax1.set_xlabel('Date')
ax1.set_ylabel('Close Price')
ax1.set_title('Apple and Samsung Close Price in 2022')
ax1.legend()


# Plot the data for Nasdaq on the right subplot
ax2.plot(nasdaq.index, nasdaq['Close'], label='Nasdaq', color = 'g')
ax2.set_xlabel('Date')
ax2.set_ylabel('Close Price')
ax2.set_title('Nasdaq Close Price in 2022')
ax2.legend()

# Show the plot
plt.show()

###########################     VERSION 2     ###########################
import urllib.request as req  # url 가져오기 
from bs4 import BeautifulSoup # html parsing 도구

from re import findall, match, sub, search  # 정규표현식 
import matplotlib.pyplot as plt 


# 0. 검색하고자하는 종목 검색 : 삼성전자, 네이버
tickers = ['%BB%EF%BC%BA%C0%FC%C0%DA']

'''
['2021-01-08','2021-01-18','2021-01-26','2021-02-15','2021-02-26',
         '2021-08-03','2021-08-13','2021-08-24','2021-08-31','2021-10-12',
         '2021-11-22','2021-12-01','2022-01-28','2022-02-10','2022-02-24',
'''

# 1. 수집 기간 날짜 설정 
#dates = [['2021-01-06','2021-01-08'],['2021-01-16','2021-01-18'],['2021-01-24','2021-01-26']]
dates = [['2021-01-06','2021-01-08'],['2021-01-16','2021-01-18']]
# 각 페이지당 url마지막 페이지 번호 추출 함수
def getLastPage(ticker, date) :
    last_page_list = []
    for date in dates :  
        try:
           print('검색중 : ', ticker)
           # 1) url 구성 
           url = f"https://finance.naver.com/news/news_search.naver?rcdate=&q={ticker}&x=0&y=0&sm=all.basic&pd=4&stDateStart={date[0]}&stDateEnd={date[1]}"
           
           # 2) url 요청          
           res = req.urlopen(url)
           data = res.read()
           
           # 3) html 파싱
           #src = data.decode('utf-8') 
           html = BeautifulSoup(data, 'html.parser')
           select_page = html.select('td[class="pgRR"] > a')
           
           page_a= select_page[0].get('href')
           
           # = 기준 split 해서 0,1 로 찾아도 된다
           search_page = search("page=.*", page_a)
           page_num = search(r'\d+', search_page.group())
           page_num = int(page_num.group()) 
           
           last_page_list.append(page_num)
         
        except Exception as e:
            print('에러 :',e)
    return last_page_list

page_list = []
for ticker in tickers : 
    page_list.extend(getLastPage(ticker, dates))
    
print(page_list)
page_list[0]

# 2. Crawler 함수(페이지, 검색날짜, 페이지 리스트) 
def crawler_func(ticker, date, page_list) :  
    print('수집중 날짜:', dates)
    print(date[0],date[1])
    day_news = []
    for page_num in page_list :    
        for curr_page in range(1, page_num + 1) : 
            print(curr_page)
            # 1) url 구성 
            url = f"https://finance.naver.com/news/news_search.naver?rcdate=&q={ticker}&x=0&y=0&sm=all.basic&pd=4&stDateStart={date[0]}&stDateEnd={date[1]}&page={curr_page}"
            
            # 2) url 요청          
            res = req.urlopen(url)
            data = res.read()
            
            # 3) html 파싱
            html = BeautifulSoup(data, 'html.parser')
            
            # 4) a 태그 수집    
            links = html.select('dl[class="newsList"] > dd') # 1) dd tag 수집
            #print(links)                
            page_news = [] # 1 페이지 news 저장
            for i,link in enumerate(links) :
                print(i,link)
                #print(link)
                news = str(link.text).strip() # 문장 끝 불용어 제거 
                page_news.append(news)  
                #print(page_news)         
            day_news.extend(page_news) # {day}의 모든 페이지 루프 돌고 day_news에 저장   
        #print(page_news)       
    return day_news
        
            
# 3. 클로러 함수 호출 


#zipped = list(zip(tickers, dates, page_list))
#print(zipped)
'''
for x,y,z in zipped:
    print(x,y,z)
    type(x)
    type(y)
'''
# 크롤링한 데이터 전처리
def clean_text(texts) :        
    punc_str = '[,.?!:;]'
    texts_re = [sub(punc_str, '', st) for st in texts]
    spec_str = '[@#$%^&*()]'
    texts_re2 = [sub(spec_str, '', st) for st in texts_re]
    texts_re3 = [sub('[a-z|A-Z]', '', st) for st in texts_re2]
    texts_re4 = [sub('\s+', ' ', st) for st in texts_re3] #형식 1 
    texts_re5 = [sub('[0-9]', '', st) for st in texts_re4]
    return texts_re5


article_dict = {}

for date in dates:
    key = (date[0], date[1])
    value = clean_text(crawler_func(tickers[0], date, page_list))
    article_dict[key] = value
 
print(article_dict)
len(article_dict.keys())
len(article_dict.values())

###########################     FUNCTIONS     ###########################

#변동 사항제일큰 데이터 N개 뽑아주는 함수
def top_N_changes(finance_data, N):
    finance_data['Abs_Change'] = finance_data['Change'].abs()
    top_N = finance_data.sort_values(by='Abs_Change', ascending=False).head(N)
    # Drop the 'Abs_Change' column
    top_N = top_N.drop('Abs_Change', axis=1)
    return top_N

print(top_N_changes(samsung, 11))



{'%BB%EF%BC%BA%C0%FC%C0%DA': ['KAIST 발열 40 낮춘 초고해상도 마이크로 LED 기술 구현', '마이크로 LED는 OLED 대비 우수한 밝기 명암비 수명이라는 장점이 있어 활발히 연구되고 있으며 삼성전자는 지난 2018년에 ‘T W 부산일보 | 2023-03-22 2231', '‘반도체 한파’에 실적 악화…대기업 작년 4분기 영업이익 70 급감', '친환경차와 스포츠유틸리티차SUV 등 인기로 현대차와 기아를 비롯한 자동차·부품 업계의 영업이익은 증가한 반면 반도체가 한파를 겪으며 삼성전 경향신문 | 2023-03-22 2229', "'AI 테마' 국내 상장사 12곳 총정리 I 주가는 왜 폭등했을까 [넘버스]", '아무래도 삼성전자가 투자했다는 게 알려지며 투자자 심리를 크게 자극한 듯 보입니다 이 회사는 한국과학기술원 휴머노이드로봇연구센터 소속 연구원들 블로터 | 2023-03-22 2224', '이에 따라 중국에 각각 낸드플래시와 D램 공장을 운영 중인 삼성전자와 SK하이닉스는 규제 대상이 됐습니다 10년간 5 이상 생산량을 늘릴 수 YTN | 2023-03-22 2220', '‘킹산직 맞네’…SK이노 평균 연봉 1억5300만원 삼성전자·하이닉스 제쳐', '또 삼성전자1억3500만 원 SK하이닉스1억3385만 원 LG전자1억1200만 원 등 다른 연봉 1억 원대 대기업보다 많은 편에 문화일보 | 2023-03-22 2219', '무려 357억 받았다 작년 연봉킹은 누구', '이재용 삼성전자 회장은 2017년부터 급여와 연봉을 받지 않는 ‘무보수 경영’을 이어가고 있다 2021년 182억5970만원을 받은 신동빈 롯 조선일보 | 2023-03-22 2211', '규정안에 삼성전자와 SK하이닉스 등 국내 반도체업체들은 일단은 안도하는 모습이다 이번 세부 규정은 향후 10년 동안 중국 공장의 웨이퍼 투입량을 5첨단 반도체 경향신문 | 2023-03-22 2209', '직원 평균 연봉 15억… 삼성전자·하이닉스 제친 기업은', '삼성전자

# 규진님


In [ ]:
# -*- coding: utf-8 -*-
"""
Kyujin Choi Project
"""

import FinanceDataReader as fdr
import pandas as pd


def Finace_data(code, name):
    # 데이터 얻기
    data = fdr.DataReader(code, start='2022-01-01', end='2022-12-31')
    # 결측지 제거
    data.dropna(inplace=True)
    # csv로 저장
    path = r'C:\ITWILL\3_TextMining'
    data.to_csv(f'{path}/df_{name}_data.csv')

#함수 실행
Finace_data('005930', 'Samsung')

########################################################################################################################################

# CSV 파일 불러오기

path = r'C:\ITWILL\3_TextMining'
df = pd.read_csv(f'{path}/df_Samsung_data.csv')

print(df.iloc[:, 0].tolist()) # index 칼럼인 Date 만 불러오기
print(dates)
#자료가 있는 날짜만 기준으로 뽑혔으니 이 날짜로만 url 생성
len(dates) # 246

# url 생성기
import urllib.parse
text_inputs = ["삼성전자"]

results = []
for text in text_inputs:
    encoded_string = text.encode('euc-kr')
    encoded_result = encoded_string.hex().upper()
    encoded_bytes = bytes.fromhex(encoded_result)
    encoded_string = encoded_bytes.decode('euc-kr')
    encoded_string = urllib.parse.quote(encoded_string, encoding='euc-kr')
    results.append(encoded_string)

print(results)
#      %BB%EF%BC%BA%C0%FC%C0%DA


urls = []
def url_maker(dates):
    for date in (dates):
        url = f"https://finance.naver.com/news/news_search.naver?rcdate=&q=%BB%EF%BC%BA%C0%FC%C0%DA&x=0&y=0&sm=all.basic&pd=4&stDateStart={date}&stDateEnd={date}"
        urls.append(url)

#함수 실행
url_maker(dates)

print(urls)
len(urls) # 246

########################################################################################################################################

#뉴스 크롤링으로 날짜별 뉴스를 모으기

import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

def scrape_news(date, url):
    subjects = []
    summaries = []
    news_dates = []

    for page in range(1, 3):
        response = requests.get(f"{url}&page={page}")
        soup = BeautifulSoup(response.content, "html.parser")

        for subject, summary in zip(soup.find_all("dd", class_="articleSubject"), soup.find_all("dd", class_="articleSummary")):
        
            subject_str = re.sub('[^a-zA-Z가-힣\s]', '', subject.text.replace('\n', '').replace('\t', ''))
            subjects.append(subject_str)

            summary_str = re.sub('[^a-zA-Z가-힣\s]', '', summary.text.replace('\n', '').replace('\t', ''))
            summaries.append(summary_str)

            news_dates.append(date)

    if len(subjects) == len(summaries):
        data = {"dates": news_dates, "subjects": subjects, "summaries": summaries}
        df = pd.DataFrame(data, index=None)
        path = r"C:\ITWILL\3_TextMining"
        df.to_csv(f"{path}/samsung_news_crawl.csv", mode='a',header=False, index=False)

for date, url in zip(dates, urls):
    scrape_news(date, url)

########################################################################################################################################

# 파일 불러오기
path = r"C:\ITWILL\3_TextMining"
df_crawl = pd.read_csv(f'{path}/samsung_news_crawl.csv')

# 칼럼 추가
df_crawl.columns = ['Date', 'Title', 'Summary']
df_crawl['Sentence'] = df_crawl['Title'] + ' ' + df_crawl['Summary']
df_crawl = df_crawl.drop(columns=['Title', 'Summary'])

# 날짜별로 그룹
grouped = df_crawl.groupby(df_crawl.columns[0], as_index=False).agg({'Sentence': ' '.join})
grouped.to_csv(f'{path}/samsung_newscrawl_grouped.csv', index=False)

########################################################################################################################################

#구글 번역 라이브러리로 한글을 영어로 번역하기

from googletrans import Translator
import pandas as pd

# 파일 불러오기
path = r"C:\ITWILL\3_TextMining"
df_grouped = pd.read_csv(f'{path}/samsung_newscrawl_grouped.csv')

translator = Translator()

# Sentence 칼럼 번역하기
df_grouped['Sentence'] = df_grouped['Sentence'].apply(lambda x: translator.translate(x, src='ko', dest='en').text)

# 파일 저장하기
df_grouped.to_csv(f'{path}/samsung_newscrawl_translated.csv', index=False)

########################################################################################################################################

# Affin 사용하여 날짜별 점수를 모으기
import pandas as pd
from afinn import Afinn

path = r"C:\ITWILL\3_TextMining"
df_translated = pd.read_csv(f'{path}/samsung_newscrawl_translated.csv')

afinn = Afinn()

# Add a new column "Afinn" with the Afinn scores
df_translated['Afinn'] = df_translated['Sentence'].apply(lambda x: afinn.score(x))

# Afinn 점수가 40점 이상이면 긍정, 미만이면 부정으로 나타냄
df_translated['Pos_Neg'] = df_translated['Afinn'].apply(lambda x: 'Positive' if x >= 40 else 'Negative')

# Drop the "Sentence" column
df_translated = df_translated.drop(columns=['Sentence'])

df_translated.to_csv(f'{path}/samsung_newscrawl_afinn_scores.csv', index=False)

########################################################################################################################################

#파일을 하나의 데이터프레임으로 만들기
path = r'C:\ITWILL\3_TextMining'

# 삼성 데이터 불러옴
df = pd.read_csv(f'{path}/df_Samsung_data.csv')
df_affin_scores = pd.read_csv(f'{path}/samsung_newscrawl_afinn_scores.csv')

# 두 파일 합치기
merged_df = pd.merge(df, df_affin_scores, on='Date')
merged_df.to_csv(f'{path}/samsung_merged.csv', index=False) # index 열 제외

########################################################################################################################################

#정확도 예측하기 

path = r'C:\ITWILL\3_TextMining'

df_final = pd.read_csv(f'{path}/samsung_merged.csv')

# 긍정과 부정예측이 맞는 부분 갯수 세기
df_final['Pos_Count'] = ((df_final['Change'] > 0) & (df_final['Pos_Neg'] == 'Positive')).astype(int)
df_final['Neg_Count'] = ((df_final['Change'] < 0) & (df_final['Pos_Neg'] == 'Negative')).astype(int)

(df_final["Pos_Count"].sum() + df_final["Neg_Count"].sum()) / len(df_final["Change"]) * 100      # 55.6910569105691

'''
긍정 예측 성공: 22
부정 예측 성공: 115
total value : 246

정확도 : (22+115) / 246 * 100 = 55.6910569105691 %
'''

########################################################################################################################################

#Afinn 과 Change 변수의 상관분석

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

path = r'C:\ITWILL\3_TextMining'
df_final = pd.read_csv(f'{path}/samsung_merged.csv')
df_final.info()


#산점도 (Scatter plot)
Afinn = df_final.Afinn
Change = df_final.Change

plt.scatter(Afinn, Change, alpha = 0.5)
plt.title('Scatter Afinn Change')
plt.xlabel('Afinn')
plt.ylabel('Change')
plt.show()

#공분산
np.cov(Afinn, Change)[0,1] # 0.02332267589241042

#상관계수
np.corrcoef(Afinn, Change)[0,1]   # 0.07930514646928592

#상관계수 그래프
corr = df_final[["Afinn", "Change"]].corr()
sns.heatmap(corr, annot=True, cmap="coolwarm")

#피어슨 계수
import scipy.stats as stats
stats.pearsonr(Afinn, Change)   # PearsonRResult(statistic=0.07930514646928591, pvalue=0.21517130626461337)


########################################################################################################################################

# 긍정 부정값이 일치하는 행만 남기고 나머지를 drop

path = r'C:\ITWILL\3_TextMining'
df_final = pd.read_csv(f'{path}/samsung_merged.csv')

df_final['Pos_Count'] = ((df_final['Change'] > 0) & (df_final['Pos_Neg'] == 'Positive')).astype(int)
df_final['Neg_Count'] = ((df_final['Change'] < 0) & (df_final['Pos_Neg'] == 'Negative')).astype(int)

df_final_filtered = df_final[(df_final['Pos_Count'] != 0) | (df_final['Neg_Count'] != 0)]
df_final_filtered.to_csv(f'{path}/samsung_filtered_data.csv', index=False)

########################################################################################################################################

#필터된 항목으로 다시 상관분석 진행

path = r'C:\ITWILL\3_TextMining'
df_final_filtered = pd.read_csv(f'{path}/samsung_filtered_data.csv')


#산점도 (Scatter plot)
Afinn = df_final_filtered.Afinn
Change = df_final_filtered.Change

plt.scatter(Afinn, Change, alpha = 0.5)
plt.title('Scatter Afinn Change')
plt.xlabel('Afinn')
plt.ylabel('Change')
plt.show()

#공분산
np.cov(Afinn, Change)[0,1] # 0.13720858217816367

#상관계수
np.corrcoef(Afinn, Change)[0,1]   # 0.61436281454109

#상관계수 그래프
corr = df_final[["Afinn", "Change"]].corr()
sns.heatmap(corr, annot=True, cmap="coolwarm")


#피어슨 계수
import scipy.stats as stats
stats.pearsonr(Afinn, Change) 
# PearsonRResult(statistic=0.61436281454109, pvalue=1.411286245756649e-15)

########################################################################################################################################



# 감정분석


# 새 섹션

감정 사전 준비

In [ ]:
!pip install afinn

데이터 준비

In [ ]:
from sklearn.datasets import fetch_20newsgroups
newsdata = fetch_20newsgroups(subset = 'train')
newsdata.data[0]

감정 상태 분류 및 시각화
  



*  감정 사전을 구성하고 감정 스코어를 측정
*  afinn 라이브러리는 감정 사전과 더불어 편리하게 감정가를 계산할 수 있는 함수를 제공



In [ ]:
from afinn import Afinn

afinn = Afinn()
for i in range(10):
  print(afinn.score(newsdata.data[i]))

모든 뉴스에 대한 감정을 시각화
긍정과 부정에 대한 갯수를 시각화

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')

positive = 0
neutral = 0
negative = 0

for i in newsdata.data:
  score = afinn.score(i)
  if score > 0:
    positive +=1
  elif score == 0 :
    neutral +=1
  else :
    negative +=1

plt.bar(np.arange(3), [positive,neutral,negative])
plt.xticks(np.arange(3), ['positive','neutral','negative'])    
plt.show()


# 삼성 주가 데이터 가져오기


In [ ]:
##############################################################################
# 삼성 vs 애플 & 나스닥 지수

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas_datareader import data as pdr
import yfinance as yfin

import matplotlib.pyplot as plt
import matplotlib.dates as mdates

yfin.pdr_override()

# 날짜 설정


tickers = ['005930.KS']
mydata = pd.DataFrame()

# 주식데이터 가져오기
samsung = pdr.get_data_yahoo(tickers[0], start='2021-01-05', end='2022-02-24')
#samsung['Adj Close'].plot(figsize = (8,5))

#samsung['Adj Close'].plot(figsize=(8,5))

###################수익율 계산######################
samsung['simple_return'] = (samsung['Adj Close']/samsung['Adj Close'].shift(1)) -1
samsung
#samsung['Adj Close'].plot(figsize = (8,5), title = '주가')
#samsung['simple_return'].plot(figsize = (8,5), title = '수익율')
section_return = samsung['simple_return'].mean()*3
print(section_return)

#변동 사항제일큰 데이터 N개 뽑아주는 함수
def top_N_changes(finance_data, N):
    finance_data['Abs_Change'] = finance_data['simple_return'].abs()
    top_N = finance_data.sort_values(by='Abs_Change', ascending=False).head(N)
    # Drop the 'Abs_Change' column
    top_N = top_N.drop('Abs_Change', axis=1)
    return top_N


samsung['simple_return'].plot(figsize = (8,5))
top30 = top_N_changes(samsung, 36)
top30.sort_index()

dates = ['2021-01-08','2021-01-18','2021-01-26','2021-02-15','2021-02-26',
         '2021-08-03','2021-08-13','2021-08-24','2021-08-31','2021-10-12',
         '2021-11-22','2021-12-01','2022-01-28','2022-02-10','2022-02-24',
         '2022-03-07','2022-04-13','2022-04-29','2022-06-13','2022-07-07',
         '2022-07-18','2022-09-13','2022-10-04','2022-10-24','2022-10-28',
         '2022-10-31','2022-11-08','2022-11-11','2022-11-30','2022-12-02']



[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,simple_return
Date,,,,,,,
2021-01-05,81600.0,83900.0,81600.0,83900.0,80414.453125,35335669,NaN
2021-01-06,83300.0,84500.0,82100.0,82200.0,78785.070312,42089013,-0.020262
2021-01-07,82800.0,84200.0,82700.0,82900.0,79455.992188,32644642,0.008516
2021-01-08,83300.0,90000.0,83000.0,88800.0,85110.875000,59013307,0.071170
2021-01-11,90000.0,96800.0,89500.0,91000.0,87219.476562,90306177,0.024775
...,...,...,...,...,...,...,...
2022-02-17,75000.0,75600.0,74500.0,75000.0,73197.109375,13605382,0.002674
2022-02-18,74600.0,74800.0,73700.0,74300.0,72513.937500,10122226,-0.009333
2022-02-21,74300.0,74300.0,74300.0,74300.0,72513.937500,0,0.000000


In [ ]:
urls = [
 'https://finance.naver.com/news/news_search.naver?rcdate=&q=%BB%EF%BC%BA%C0%FC%C0%DA&x=0&y=0&sm=all.basic&pd=4&stDateStart=2022-01-01&stDateEnd=2022-01-01', 
 'https://finance.naver.com/news/news_search.naver?rcdate=&q=%BB%EF%BC%BA%C0%FC%C0%DA&x=0&y=0&sm=all.basic&pd=4&stDateStart=2022-01-02&stDateEnd=2022-01-02'
]
afinn = Afinn()

v_list = []  # define v_list outside the function

def scrape_news(url):
    subjects = []
    summaries = []

    for page in range(1, 3):
        response = requests.get(f"{url}&page={page}")
        soup = BeautifulSoup(response.content, "html.parser")

        for subject, summary in zip(
            soup.find_all("dd", class_="articleSubject"),
            soup.find_all("dd", class_="articleSummary")
        ):
            # Remove special characters from subjects and summaries
            subject_str = subject.text.replace('\n', '').replace('\t', '').replace('\r', '').strip()
            subject_str = ''.join(c for c in subject_str if c.isalnum() or c.isspace())
            subjects.append(subject_str)

            summary_str = summary.text.replace('\n', '').replace('\t', '').replace('\r', '').strip()
            summary_str = ''.join(c for c in summary_str if c.isalnum() or c.isspace())
            summaries.append(summary_str)

    positive = 0
    neutral = 0
    negative = 0
    
    total = subjects + summaries
    okt = Okt()
    nouns = []
    for sent in total:
        for noun in okt.nouns(sent):
            if len(noun) > 1 and len(noun) <= 10: # Only select nouns with length between 2 and 10
                nouns.append(noun)
   
    translator = Translator()
    nouns_t = []
    for t in nouns:
        t = translator.translate(t, dest='en').text
        nouns_t.append(t)
    
    
    for n in nouns_t:
        score = afinn.score(a)
        if score > 0:
            positive +=1
        elif score == 0 :
            neutral +=1
        else :
            negative +=1
    if positive + negative == 0:
        v = 0
    else:
        v = positive/(positive + negative)
    try:      
        v_list.append(v)
    except:
        v_list.append(0)
        

for url in urls:
    scrape_news(url)

print(v_list)   

NameError: ignored

# 주식데이터 분석


In [ ]:

import FinanceDataReader as fdr
import pandas as pd

# Define the ticker symbol for Samsung
symbol = '005930'

# Define the date ranges for the average returns
date_ranges = [('2021-01-06', '2021-01-08'),
               ('2021-01-16', '2021-01-18'),
               ('2021-01-24', '2021-01-26'),
               ('2021-02-13', '2021-02-15'),
               ('2021-02-24', '2021-02-26'),
               ('2021-11-20', '2021-11-22'),
               ('2021-11-29', '2021-12-01'),
               ('2022-01-26', '2022-01-28'),
               ('2022-02-08', '2022-02-10'),
               ('2022-02-22', '2022-02-24')]

# Get the stock price data from FRED
df = fdr.DataReader(symbol, '2021-01-01', '2022-03-31')

# Calculate the average return for each date range
avg_return = {}
for start_date, end_date in date_ranges:
    temp_df = df[(df.index >= start_date) & (df.index <= end_date)]
    avg = temp_df['Change'].fillna(0).mean()*3
    avg_return[(start_date, end_date)] = {'return': avg}

print(avg_return)


# Update the data dictionary with the average return rates
for start_date, end_date in date_ranges:
    key = (start_date, end_date)
    data[key].update(avg_return[key])

print(data)

import pandas as pd
import matplotlib.pyplot as plt


sent_scores = []
returns = []
for key in data:
    sent_scores.append(data[key]['sent_score'])
    returns.append(data[key]['return'])

plt.scatter(sent_scores, returns)
plt.xlabel('Sentiment Score')
plt.ylabel('Return')
plt.title('Sentiment Score vs Return')
plt.show()


from scipy.stats import pearsonr

# 감정분석 스코어 리스트와 수익율 리스트 (데이터 사이즈 같아야함)
sent_scores = [d['sent_score'] for d in data.values()]
returns = [d['return'] for d in data.values()]

# 피어슨 상관계수(corr)와 and p값 (p-value) 계산   https://m.blog.naver.com/sub_om/221828230101 블로그 내용 참고
corr, pval = pearsonr(sent_scores, returns)

print("Pearson correlation coefficient:", corr) #0.3~0.7 : 뚜렷한 상관 관계 , 0.7~ 1.0 : 강한 상관관계
print("p-value:", pval) # 0.05 이하면 Null Hypothesis를 reject -> 두 데이터 사이에 상관관계가 있음을 증명

# 시각화



In [ ]:

# Scatter 1
import matplotlib.pyplot as plt
df_final.plot(kind='scatter', x='positive', y='profit')
plt.xlabel('net positive value')
plt.ylabel('profit')
plt.title('Positive vs Profit Scatter Plot')
plt.show()

# Scatter 2
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style("darkgrid")
plt.figure(figsize=(8,6))
sns.scatterplot(data=df_final, x="positive", y="profit")
plt.title("Positive Sentiment vs Profit")
plt.xlabel("Positive Sentiment")
plt.ylabel("Profit")
plt.show()

# Line plot
import matplotlib.pyplot as plt
plt.figure(figsize=(8,6))
plt.plot(df_final["date"], df_final["positive"], label="Positive Sentiment")
plt.plot(df_final["date"], df_final["profit"], label="Profit")
plt.title("Positive Sentiment and Profit over Time")
plt.xlabel("Date")
plt.xticks(rotation=45)
plt.ylabel("Value")
plt.legend()
plt.show()

# Bar plot
df_final.plot.bar(x='date', y=['positive', 'profit'], position=0.5, width=0.4, figsize=(12, 6))
plt.title('Positive vs Profit')
plt.xlabel('Date')
plt.ylabel('Value')
plt.show()